In [29]:
import modal
from egfr_binder_rd2.utils import hash_seq, load_evolution_metadata
from egfr_binder_rd2 import EGFR, EGFS, OFFICIAL_EGFR
import pandas as pd
from pathlib import Path
from egfr_binder_rd2 import MODAL_VOLUME_PATH, OUTPUT_DIRS
from egfr_binder_rd2.fold import get_a3m_path
import seaborn as sns
from egfr_binder_rd2.solubility import calculate_solubility
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

get_msa = modal.Function.lookup("simplefold", 'get_msa_for_binder')
# a3m_from_template = modal.Function.lookup("simplefold", 'a3m_from_template')
fold_binder = modal.Function.lookup("simplefold", 'fold_binder')
update_metrics = modal.Function.lookup("simplefold", 'update_metrics_for_all_folded')
esm2_pll = modal.Function.lookup("esm2-inference", 'process_sequences')
esm2_pll_exact = modal.Function.lookup("esm2-inference", 'process_sequences_exact')
update_pll_metrics = modal.Function.lookup("esm2-inference", 'update_pll_metrics')
get_exact_plls = modal.Function.lookup("esm2-inference", 'update_exact_pll_metrics')
sample_sequences = modal.Function.lookup("bt-training", 'sample_sequences')
update_high_quality_metrics = modal.Function.lookup("simplefold", 'update_high_quality_metrics')
fold_high_quality = modal.Function.lookup("simplefold", 'fold_high_quality')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
hash_seq(f'SYEGYCENGGTLQHIESLDSYTCKCLKGYTGDRCQSQDLRYLYLEGGEGGEGGEGGEGGEGGRGGEGGRGGEGGRGGEGGRGGEGGEGGRGGEGGEGGEGGEGGEGGRGGEGGRGGEGGRGGEGGRGGEGGEGGRGGEGREGGEGGEGGEGGRGGEGGRGGEGGRGGEGGRGGEGGEGGRSYEGYCENGGTLQHIESLDSYTCKCLKGYTGDRCQSQDLRYLYLE')

'c2a9f3'

In [4]:

seqs = [
    'RLFSKCPRRYEGICKNNGTCKYAINLRTYTCRCLSGYTGDRCEELDIRYLLLLY',
]
result = get_msa.remote(seqs)
print(result)

result = fold_high_quality.remote(result)
print(result)

[PosixPath('/colabfold_data/msa_results/861628.a3m')]


TypeError: unsupported operand type(s) for /: 'PosixPath' and 'list'

In [27]:
seqs = [
    # 'RLFSKCPRRYEGICKNNGTCKYAINLRTYTCRCLSGYTGDRCEELDIRYLLLLY',
    # 'SYEGYCENGGTCVHVEALDSYTCKCLKGYTGDRCQSQDLRYLLLE',
    # 'NLFSRCPKRYAGICENNGKCRYAINLRTYTCICKSGYTGDRCQELDIRYLLLLN',
    # 'SYEGYCLNGGECRHVKSLDTYTCKCEAGYTGDRCQTRDLRYLELL',
    # 'EGHKECPRRYRGICENNGSCRYASNLDTYTCQCLEGYTGARCQELDIRY',
    # 'CPARYNGICTNEGRCQYARNLETYTCQCLTGYTDHRCEELDIRY',
    # 'CPARYNGICLNEGRCQYVRNLETYTCKCLTGYTDHRCEELDIRY',
    #  'NLFSKCPRRYRGICKNNGKCKYAINLRTYTCICVSGYTGERCQELDIRYLLLRY',
    #  'SRFSNCPRRYRGICTNSGECTYAKNLRTYTCQCVSGYTGHRCEELDIRYL',
    # 'SRFSNCPRRYRGICTNNGECTYAKNLRTYTCQCVSGYTGARCQELDIRYL',
    # 'SRFSNCPRRYRGICANNGECTYAKNLRTYTCQCVSGYTGARCQELDIRYL',
    # 'SRFSNCPRRYRGICANNGECTYAKNLRTYTCQCVSGYTGDRCQELDIRYL',
    # 'SYEGYCLNGGTLVHVESLDSYTCGCLKGYTGDRCQSQDLRYLYLE',
    # 'SYEGYCENGGTLQHIESLDSYTCKCLKGYTGDRCQSQDLRYLYLE',
    # 'YSFTQCPSSYDGICMNGGVCRHVLSLDSYTCRCKEGYIGDRCETLDLRYLELR',
    # 'SYEGYCENGGTLQHIESLDSYTCKCLKGYTGDRCQSQDLRYLYLE',
    # 'CPARYNGICTNHGRCQYARNLRTYTCQCLPGYTKHRCQELDIRY',
    # 'SYNGYCLNNGRCQHIMSLDSYTCRCEVGYSGDRCQTHDLRWLELR',
    # 'HYDGYCLNGGACRRIESLHSYTCQCQKGYSGDRCQTRDLRWLELQN'
    # 'RYDGYCLDGGVCKHIESLDKYTCECVIGYTGDRCQTRDLRWLELR',
    # 'EYDGYCLNGGVCMHIESLDKYTCECVIGYTGDRCQTRDLRWLELR',
    # 'PYDGYCLNGGVCMHIESLDKYTCICVIGYTGDRCQTRDLRWLELR',
    # 'PYDGYCLNGGRCTAIESLDKYTCVCRAGYTGERCQTRDLRWLELR',
    # 'PYDGYCLNGGECMHIFSLDKYTCICVGGYTGDRCQTRDLRWLELR',
    # 'SYDGKCLNDGACRYIESLDSYTCHCVSGYTGDRCQTRDLRWLELR',
    # 'SNCPRRYRGICENNGSCKYRENLRTYTCQCRSGYTGARCQELDIRYLL',
    # 'SYEGYCENGGTLVHVESLDSYTCGCLKGYTGDRCQSQDLRYLYLE',
    # 'TYDGYCLNGGKCEHVESLDKYTCNCVSGYTGDRCQERDLRWLEHR',
    # 'SNCPRRYRGICENNGSCKYATNLRTYTCQCLSGYTGARCQELDIRYLL'
    # 'NLFSRCPKRYHGICENNGQCRYAINLRTYCCECKSGYTGDRCQELDIRYLLLLN',
    # 'NLFSRCPKRYHGICENNGQCRYAHNLRTYTCVCDSGYTGDRCQELDIRYLLLLN',
    # 'NLFKRCPKRYHGICENNGQCRYAINLRTYCCICHSGYTGDRCQELDIRYLLLLN',
    # 'YLFEKCPYKFKGICNNNGVCRYAINLRKYTCICLEGYTGDRCQEIDIRYLLLQL',
    # 'NKFSACPRRYRGICTNGGHCQYAHNLRTYTCQCRSGLTGYRCQELDIRYLLL',
    # 'YNFSRCPKRYHGICGNNGQCRYAINLRTYTCRCVSGYTGDRCCELDIRYLLLLN',
    'NLFSRCPKRYHGICENNGQCRYAHNLRTYTCVCDSGYTGDRCQELDIRYLLLLN',
    'YLFEKCPYKFKGICNNNGVCRYAINLRKYTCICLEGYTGDRCQEIDIRYLLLQL',
    'NLFKRCPKRYHGICENNGQCRYAINLRTYCCICHSGYTGDRCQELDIRYLLLLN',
    'SYDGKCLNNGACRYIERLDSYTCHCVSGYTGDRCQTRDLRWLELR',
    'KYDGYCNNNGVCHHIESLDKYTCNCRVGYSGDRCQTRDLRWLELRY',
    'SYDGYCNNHGVCRHIESLDSWTCQCRQGYEGDRCQTRDLRWLELN',
    'SRFSNCPRRYRGICTNNGECTYAKNLRTYTCQCVSGYTGHRCQELDIRYL',
]

In [28]:
template = """source .venv/bin/activate
cd src/egfr_binder_rd2/
modal run fold.py::fhq --s {seq}
"""

for seq in seqs:
    print(template.format(seq=seq))


source .venv/bin/activate
cd src/egfr_binder_rd2/
modal run fold.py::fhq --s NLFSRCPKRYHGICENNGQCRYAHNLRTYTCVCDSGYTGDRCQELDIRYLLLLN

source .venv/bin/activate
cd src/egfr_binder_rd2/
modal run fold.py::fhq --s YLFEKCPYKFKGICNNNGVCRYAINLRKYTCICLEGYTGDRCQEIDIRYLLLQL

source .venv/bin/activate
cd src/egfr_binder_rd2/
modal run fold.py::fhq --s NLFKRCPKRYHGICENNGQCRYAINLRTYCCICHSGYTGDRCQELDIRYLLLLN

source .venv/bin/activate
cd src/egfr_binder_rd2/
modal run fold.py::fhq --s SYDGKCLNNGACRYIERLDSYTCHCVSGYTGDRCQTRDLRWLELR

source .venv/bin/activate
cd src/egfr_binder_rd2/
modal run fold.py::fhq --s KYDGYCNNNGVCHHIESLDKYTCNCRVGYSGDRCQTRDLRWLELRY

source .venv/bin/activate
cd src/egfr_binder_rd2/
modal run fold.py::fhq --s SYDGYCNNHGVCRHIESLDSWTCQCRQGYEGDRCQTRDLRWLELN

source .venv/bin/activate
cd src/egfr_binder_rd2/
modal run fold.py::fhq --s SRFSNCPRRYRGICTNNGECTYAKNLRTYTCQCVSGYTGHRCQELDIRYL

